## 2 Feb - Resume Parser

**This notebook is to run trials of my codes for the resume parser app.**

In [1]:
# Import necessary libraries
import pandas as pd
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from PyPDF2 import PdfReader 

In [2]:
# Load NLP
nlp = spacy.load('en_core_web_md')

In [3]:
# Load the sample resume
reader = PdfReader('someone_cv.pdf')

In [4]:
# Extract text from the resume
text = str()
for i in range(len(reader.pages)):
    page = reader.pages[i]
    if i == 0:
        text += page.extract_text()
    else:
        text += ' ' + page.extract_text()

text[:1000]

'Y oshua Bengio - C.V .\nDépartement d’informatique\net recherche opérationnelle,\nUniversité de Montréal\nPhone : 514-343-6804\nYoshua.Bengio@mila.quebec\nhttps://mila.quebec/en/yoshua-\nbengio/A.M. Turing Award,\nFellow of the Royal Society of London,\nFellow of the Royal Society of Canada,\nCIFAR AI Chair\nTitles and Distinctions\n\x0fFull Professor , Université de Montréal, department of computer science and operations\nresearch, since 2002. Recruited as assistant prof. in 1993. Previously associate professor\n(1997-2002) and assistant professor (1993-1997).\n\x0fCanada Research Chair onStatistical Learning Algorithms 2001-2019. Recipient\nof a CIFAR AI (CCAI) Chair (2018-2023, 1.25M$).\n\x0fRecipient of the 2018 A.M. Turing Award, of the 2019 Killam Prize, of the 2017 Marie-\nVictorin Québec Prize (highest distinction in the sciences, for the province of Québec),\nPrix d’excellence du FRQNT 2019, Medal of the 50th Anniversary of the Ministry of Inter-\nnational Relations and Franc

In [5]:
# Preprocess the text
def preprocessing(sentence):
    
    stopwords = list(STOP_WORDS)
    doc = nlp(sentence)
    cleaned_tokens = []
    
    for token in doc:
        if token.text not in stopwords and token.pos_ != 'PUNCT' and token.pos_ != 'SPACE' and \
            token.pos_ != 'SYM':
                cleaned_tokens.append(token.lemma_.lower().strip())
                
    return " ".join(cleaned_tokens)

text = preprocessing(text)

text[:1000]

'y oshua bengio c.v département d’informatique et recherche opérationnelle université de montréal phone 514 - 343 6804 yoshua.bengio@mila.quebec https://mila.quebec/en/yoshua- bengio a.m. turing award fellow royal society london fellow royal society canada cifar ai chair titles distinctions \x0ffull professor université de montréal department computer science operation research 2002 recruit assistant prof 1993 previously associate professor 1997 2002 assistant professor 1993 1997 \x0fcanada research chair onstatistical learning algorithms 2001 2019 recipient cifar ai ccai chair 2018 2023 1.25m$ \x0frecipient 2018 a.m. turing award 2019 killam prize 2017 victorin québec prize high distinction science province québec prix d’excellence du frqnt 2019 medal 50th anniversary ministry inter- national relations francophonie 2018 2019 ieee cis computational intelligence society neural networks pioneer award acfas urgel archambault 2009 prize cove- ring physics mathematic computer science engine

In [6]:
# Add entity ruler to NLP
ruler = nlp.add_pipe("entity_ruler")

In [7]:
# Load skill and education data
ruler.from_disk('skills_education.jsonl')